# Working with boundary fluxes

In [1]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from NesrHydrusAnalyst import *

## Reading the `6-BoundaryData.csv` files

In [2]:
extensions = ('', 'a', 'b', 'c', 'd', 'e')
sources = [f'../Datasets/H3D2_SandDitch0014{x}' for x in extensions]
dfs = [
    pd.read_csv(os.path.join(sources[x], 'Nesr', '6-BoundaryData.csv'))
    for x in range(len(extensions))
]
# Sample
dfs[0].head()

FileNotFoundError: [Errno 2] File b'../Datasets/H3D2_SandDitch0014\\Nesr\\6-BoundaryData.csv' does not exist: b'../Datasets/H3D2_SandDitch0014\\Nesr\\6-BoundaryData.csv'

### I found an error, that this is not the desired table for boundaries
The displayed table is for `Cum_Q.out, h_Mean.out, v_Mean.out, Run_inf.out` tables

While we need the outputs of `Boundary.out`

So, **Modify the `export_all_csvs` function**

#### The functions were modified on pyCharm

In [ ]:
# def export_all_csvs(source_folder,
#                     save_normal_grid=True,
#                     pretty_output=True,
#                     save_rotated_only=False,
#                     rotation_angle=0,  #=2.2899
#                     rotation_axis='y'):
#     # Reading the files `MESHTRIA.TXT`, `TH.TXT`, `V.TXT`, `H.TXT`
#     df = read_hydrus_data(
#         folder=source_folder, save_to_csv=False, read_velocities=True)
#     n = 0
#     if not save_rotated_only:
#         if save_normal_grid:
#             n += 1
#             save_this(df, source_folder, f'{n}-Original_Grid', authorized=True)
#         if pretty_output:
#             df_pr = reform_grid(df)
#             n += 1
#             save_this(
#                 df_pr, source_folder, f'{n}-Original_PrettyGrid', authorized=True)
#     else:
#         if rotation_angle != 0:
#             df_rotated = rotate_back(
#                 df, rotation_angle, rotation_axis=rotation_axis)
#             if save_normal_grid:
#                 n += 1
#                 save_this(
#                     df_rotated,
#                     source_folder,
#                     f'{n}-Rotated_Grid',
#                     authorized=True)
#             if pretty_output:
#                 df_rotated = reform_grid(df_rotated)
#                 n += 1
#                 save_this(
#                     df_rotated,
#                     source_folder,
#                     f'{n}-Rotated_PrettyGrid',
#                     authorized=True)

#     # If required, rotate the given dataframe

#     # Combining the two files: `A_Level.out` and `ATMOSPH.IN`
#     n += 1
#     save_this(
#         read_a_level_out(source_folder, geom='3D'),
#         source_folder,
#         f'{n}-Atm_A_level',
#         authorized=True)

#     # Saving info from one file: Balance.out
#     n += 1
#     save_this(
#         read_balance_out(source_folder)[1],
#         source_folder,
#         f'{n}-MassBalance',
#         authorized=True)

#     # Saving eight files: SELECTOR.IN, DIMENSIO.IN, Run_Inf.out, Balance.out,
#     # A_Level.out, ATMOSPH.IN, `Boundary.out` and `Check.out` files
#     n += 1
#     save_all_simulation_info(
#         source_folder, saving_name=f'{n}-Simulation_info', save_type='csv')
#     # Reading and combining Boundary.out tables
#     n += 1
#     save_this(
#         read_boundary_data(source_folder, 19, 22, dt_frame=df),
#         source_folder,
#         f'{n}-BoundaryData',
#         authorized=True)
#     # Reading and combining Cum_Q.out, h_Mean.out, v_Mean.out, Run_inf.out tables
#     n += 1
#     save_this(
#         get_mean_outs_table(source_folder),
#         source_folder,
#         f'{n}-CumulativeMeans',
#         authorized=True)
#     # Reading the file `Check.out` for materials properties
#     n += 1
#     save_this(
#         get_materials_properties(source_folder),
#         source_folder,
#         f'{n}-MaterialsData',
#         authorized=True)
#     print('**All the CSV files were exported successfully**')
#     pass

In [ ]:
# def read_boundary_data(folder='Current',
#                        titles_loc=19,
#                        data_begins=22,
#                        nums_per_line=10,
#                        save_to_csv=False,
#                        dt_frame=None):
#     '''
#     A function to read both BOUNDARY.IN and  Boundary.outfiles from HYDRUS 
#         outputs, then to:
#             1- return one dataframe contains both data in a decent format.
#             2- save this output to a CSV file (optional, True by default)
#     Input:
#         The name of the main folder (leave balank for the current folder)
#         titles_loc=19, the line number of the first line contains captions
#         data_begins=22, the line number of the first line contains data
#         nums_per_line=10, teh number of columns per line in BOUNDARY.IN file
#         The option to save_to csv, default =True (Boolean)
#     '''

#     # Specify the source folder
#     if folder == 'Current':
#         read_dir = os.getcwd()
#     else:
#         read_dir = folder
#     if dt_frame is None:
#         dt_frame= read_hydrus_data(folder=folder, 
#                              save_to_csv=False, 
#                              read_velocities=True)
#     # Finding number of nodes in the file
#     mesh_file = os.path.join(read_dir, 'BOUNDARY.IN')
#     num_cells = int(linecache.getline(mesh_file, 4).split()[0])
#     # Define dataframe titles
#     titles = ['n', 'surface_area']

#     def get_num_lines(num_cells, nums_per_line=10):
#         num_lines = int(num_cells / nums_per_line)
#         if num_cells % nums_per_line > 0: num_lines += 1
#         return num_lines

#     def read_snakey_list(file_name, start_line, end_line, data_type=float):
#         '''
#         Reading a list of numbers thar are stored in a text file, where
#         there are `numbers_count` numbers, stored as `nums_per_line` numbers 
#         per line, the starting position of first occurence of numbers is at
#         `start_line` line number
#         returns a numpy array of the data with the `data_type` given
#         '''
#         points = []
#         for i in range(start_line, end_line):
#             points.extend(linecache.getline(file_name, i).split())
#         return np.array(points, data_type)

#     num_lines = get_num_lines(num_cells, nums_per_line)

#     first_line = 10
#     end_line = first_line + num_lines
#     points = read_snakey_list(mesh_file, first_line, end_line, data_type=int)

#     first_line = end_line + 1
#     end_line = first_line + num_lines
#     areas = read_snakey_list(mesh_file, first_line, end_line)
#     data = np.array([points, areas]).T
#     data_in = pd.DataFrame(
#         data, columns=titles).astype({
#             "n": int,
#             "surface_area": float
#         })

#     # Adding the surface area to the above table
#     # print(num_cells, nums_per_line, num_lines)
#     frames = []
#     time_steps=get_available_timesteps(dt_frame)[1:]
#     # print(time_steps)
#     for t in time_steps:
#         data_ends = data_begins + num_cells - 1
#         # print(titles_loc, data_begins, num_lines, data_ends)
#         dft = get_means_table(
#             os.path.join(folder, 'Boundary.out'),
#             titles_loc,
#             data_begins,
#             units_location=None,
#             reading_end=data_ends,
#             replace_units=False)  #.sample(3)
#         # add a column for time
#         dft = dft.assign(time_step=t)
#         # convert all columns to numeric
#         cols = list(dft)
#         # print(cols)
#         # display(dft)
#         dft[cols] = dft[cols].apply(pd.to_numeric, errors='coerce')
#         dft = pd.merge(dft, data_in, on='n')
#         frames.append(dft)
#         titles_loc = data_ends + 5
#         data_begins = titles_loc + 3
#     # display(frames)
#     frames = pd.concat(frames, ignore_index=True)
    
#     # Saving if allowed
#     # save_this(dataframe, data_folder, output_name, authorized=True)
#     if save_to_csv:
#         save_this(frames, folder, 'boundary_data.CSV', authorized=save_to_csv)
    
#     return frames  

In [ ]:
read_boundary_data(sources[0], 19, 22)

In [ ]:
# export_all_csvs(sources[0])

#### Testing after pycharm modifications

## Rebuilding the csv files

In [3]:
sources = [f'../Datasets/H3D2_SandDitch0014{x}' for x in ('', 'a', 'b', 'c', 'd', 'e')]
for source in sources:
    print(f'\nFor the {source.split("/")[2][-3:]} simulation')
    export_all_csvs(source, rotation_angle=0)


For the 014 simulation
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014/Nesr/01-Original_Grid.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014/Nesr/02-Original_PrettyGrid.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014/Nesr/05-Atm_A_level.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014/Nesr/06-MassBalance.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014/Nesr/07-Simulation_info.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014/Nesr/08-CumulativeMeans.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014/Nesr/09-BoundaryData.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014/Nesr/10-MaterialsData.csv
**All the CSV files were exported successfully**

For the 14a simulation
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014a/Nesr/01-Original_Grid.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014a/Nesr/02-Original_PrettyGrid.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014a/Nesr/05-Atm_A_level.csv
**SAVED** >> path: ../Datasets/H3D2_SandDitch0014a/Nesr

## Reading the `09-BoundaryData.csv` files

In [4]:
extensions = ('', 'a', 'b', 'c', 'd', 'e')
sources = [f'../Datasets/H3D2_SandDitch0014{x}' for x in extensions]
dfs = [
    pd.read_csv(os.path.join(sources[x], 'Nesr', '09-BoundaryData.csv'))
    for x in range(len(extensions))
]
# Sample
dfs[0].head()

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step,surface_area
0,1,9,0.0,0.0,25.0,-6,0.000,100000.0,-99.97,0.243,20.0,0.00,5,0.000000
1,2,12,0.0,30.0,25.0,-6,0.000,100000.0,-108.69,0.236,20.0,0.00,5,0.000000
2,3,13,0.0,0.0,28.0,-4,0.000,0.0,-22.31,0.098,20.0,0.00,5,1.125000
3,4,14,45.0,0.0,28.0,-3,0.750,-1.0,-3.09,0.413,20.0,3.75,5,0.749999
4,5,15,45.0,30.0,28.0,-3,0.375,-1.0,-3.21,0.410,20.0,1.87,5,0.375000


![title](../Assets/Domain.png)

In [6]:
df=dfs[0]
df.head()

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step,surface_area
0,1,9,0.0,0.0,25.0,-6,0.000,100000.0,-99.97,0.243,20.0,0.00,5,0.000000
1,2,12,0.0,30.0,25.0,-6,0.000,100000.0,-108.69,0.236,20.0,0.00,5,0.000000
2,3,13,0.0,0.0,28.0,-4,0.000,0.0,-22.31,0.098,20.0,0.00,5,1.125000
3,4,14,45.0,0.0,28.0,-3,0.750,-1.0,-3.09,0.413,20.0,3.75,5,0.749999
4,5,15,45.0,30.0,28.0,-3,0.375,-1.0,-3.21,0.410,20.0,1.87,5,0.375000


In [8]:
df.describe()

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step,surface_area
count,9216.000000,9216.000000,9216.000000,9216.000000,9216.000000,9216.000000,9216.000000,9216.000000,9216.000000,9216.000000,9216.0,9216.000000,9216.000000,9216.000000
mean,384.500000,2519.195312,19.072266,15.000000,25.240234,-4.250000,0.029248,12499.981774,-24.811094,0.144934,20.0,4.310245,695.833333,1.817871
std,221.714345,705.597443,14.764025,9.076721,7.997172,0.775647,0.229427,33073.692689,14.344496,0.113755,0.0,20.759351,917.516476,0.787112
min,1.000000,9.000000,0.000000,0.000000,0.000000,-6.000000,-0.000885,-1.000000,-120.650000,0.063000,20.0,-0.749000,5.000000,0.000000
25%,192.750000,2248.750000,4.500000,7.500000,28.000000,-4.000000,0.000000,0.000000,-36.822500,0.069000,20.0,0.000000,52.500000,1.125000
50%,384.500000,2440.500000,18.000000,15.000000,28.000000,-4.000000,0.000000,0.000000,-24.880000,0.096000,20.0,0.000000,210.000000,2.250000
75%,576.250000,2632.250000,31.500000,22.500000,28.000000,-4.000000,0.000000,0.000000,-16.237500,0.159000,20.0,0.000000,900.000000,2.250000
max,768.000000,4963.000000,45.000000,30.000000,28.000000,-3.000000,2.250000,100000.000000,8.180000,0.430000,20.0,135.000000,2880.000000,2.250000


In [10]:
df_top = df[df['z']==28]
df_top

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step,surface_area
2,3,13,0.0,0.0,28.0,-4,0.000,0.0,-22.31,0.098,20.0,0.00,5,1.125000
3,4,14,45.0,0.0,28.0,-3,0.750,-1.0,-3.09,0.413,20.0,3.75,5,0.749999
4,5,15,45.0,30.0,28.0,-3,0.375,-1.0,-3.21,0.410,20.0,1.87,5,0.375000
5,6,16,0.0,30.0,28.0,-4,0.000,0.0,-21.41,0.102,20.0,0.00,5,1.500000
6,7,17,42.0,30.0,28.0,-4,0.000,0.0,-4.73,0.369,20.0,0.00,5,1.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9140,693,2749,34.5,1.5,28.0,-4,0.000,0.0,-39.58,0.066,20.0,0.00,2880,2.250000
9141,694,2750,36.0,1.5,28.0,-4,0.000,0.0,-39.56,0.066,20.0,0.00,2880,2.250000
9142,695,2751,37.5,1.5,28.0,-4,0.000,0.0,-39.55,0.066,20.0,0.00,2880,2.250000
9143,696,2752,39.0,1.5,28.0,-4,0.000,0.0,-39.54,0.066,20.0,0.00,2880,2.250000


In [11]:
df_top.describe()

,i,n,x,y,z,Code,Q,v,h,th,Temp,cumQ,time_step,surface_area
count,7812.000000,7812.000000,7812.000000,7812.000000,7812.0,7812.000000,7812.000000,7812.000000,7812.000000,7812.000000,7812.0,7812.000000,7812.000000,7812.000000
mean,368.953917,2395.130568,22.500000,15.000000,28.0,-3.935484,0.034508,-0.021505,-24.438007,0.142084,20.0,5.097844,695.833333,2.109447
std,192.668639,330.273233,13.417267,9.083532,0.0,0.245686,0.248830,0.145071,13.372069,0.118436,0.0,22.457364,917.525424,0.375469
min,3.000000,13.000000,0.000000,0.000000,28.0,-4.000000,0.000000,-1.000000,-40.240000,0.065000,20.0,0.000000,5.000000,0.375000
25%,209.000000,2265.000000,10.500000,7.500000,28.0,-4.000000,0.000000,0.000000,-37.730000,0.067000,20.0,0.000000,52.500000,2.250000
50%,372.000000,2428.000000,22.500000,15.000000,28.0,-4.000000,0.000000,0.000000,-25.510000,0.088000,20.0,0.000000,210.000000,2.250000
75%,535.000000,2591.000000,34.500000,22.500000,28.0,-4.000000,0.000000,0.000000,-16.150000,0.134000,20.0,0.000000,900.000000,2.250000
max,697.000000,2753.000000,45.000000,30.000000,28.0,-3.000000,2.250000,0.000000,8.180000,0.430000,20.0,135.000000,2880.000000,2.250000
